In [22]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME", "gpt-4.1")

In [20]:
import json
from typing import Annotated
from semantic_kernel.functions import kernel_function

# プラグイン関数の作成
class UserPlugin:
    @kernel_function(
        name="get_plan",
        description="指定されたユーザーIDから契約情報を取得します",
    )
    def get_plan(
        self, 
        user_id: Annotated[int, "ユーザーID（4桁の数字）"]
    ) -> Annotated[str, "ユーザーの契約プラン名（例: '安心保障プラン'）"]:
        
        sample_user = [
            {"id": 1234, "name": "佐藤太郎", "email": "sato@example.com", "plan": "安心保障プラン"},
            {"id": 2345, "name": "鈴木花子", "email": "hanako.suzuki@example.com", "plan": "総合生活サポートプラン"},
            {"id": 3456, "name": "田中一郎", "email": "ichiro.tanaka@example.com", "plan": "シンプルプラン"},
        ]

        for user in sample_user:
            if user["id"] == user_id:
                return json.dumps(user["plan"])
        return "該当するユーザーが見つかりませんでした。"


In [18]:
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

In [23]:
# Azure Chat Completion の初期化
azure_chat_completion = AzureChatCompletion(
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)

In [24]:
# エージェントを作成
agent = ChatCompletionAgent(
    service=azure_chat_completion,
    name="EmployeeAssistant",
    instructions=(
        "あなたはコールセンターで働く従業員向けサポートAIアシスタントです。"
        "従業員の質問に答えたり、サポートを提供したりします。"
    ),
    plugins=[UserPlugin()]
)

In [25]:
# スレッドを作成
thread = ChatHistoryAgentThread()

# レスポンスを取得
response = await agent.get_response(
    messages=["ユーザー ID 3456 のお客様の保険の加入状況を教えて。"],
    thread=thread,
)

print(response)

ユーザーID 3456のお客様は、「シンプルプラン」にご加入されています。

他にご確認されたい内容があればお知らせください。
